In [31]:
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

Read data

In [32]:
df_races = pd.read_csv('01_raw/races.csv')
df_circuits = pd.read_csv('01_raw/circuits.csv')
df_drivers = pd.read_csv('01_raw/drivers.csv')
df_results = pd.read_csv('01_raw/results.csv')
# df_cons_res = pd.read_csv('01_raw/constructor_results.csv')

In [33]:
df_results.head(1)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.000,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1


In [34]:
df_races.head(1)

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,29/3/2009,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...


In [35]:
df_circuits.head(1)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.850,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...


In [36]:
df_drivers.head(1)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton


Create master dataset

In [37]:
df_drivers = df_drivers[['driverId', 'driverRef', 'nationality']]
df_drivers.columns = ['driverId', 'driverRef', 'driver_nationality']
df_results['driverId'] = df_results['driverId'].astype(str)
df_drivers['driverId'] = df_drivers['driverId'].astype(str)
df = df_results.merge(df_drivers, on='driverId', how='left', validate='m:1')

In [38]:
df_races = df_races[['raceId', 'year', 'round', 'name', 'date', 'time', 'circuitId']]
df_races.columns = ['raceId', 'race_year', 'round', 'race_name', 'race_date', 'race_time', 'circuitId']
df['raceId'] = df['raceId'].astype(str)
df_races['raceId'] = df_races['raceId'].astype(str)
df = df.merge(df_races, on='raceId', how='left')

In [44]:
df[df['raceId'] == '966']

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,driverRef,driver_nationality,race_year,round,race_name,race_date,race_time,circuitId,circuit_name,circuit_country,circuit_alt
23308,23313,966,1,131,44,1,1,1,1,25.000,71,1:40:31.402,6031402,66,4,1:22.596,187.592,1,hamilton,British,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227
23309,23314,966,3,131,6,2,2,2,2,18.000,71,+8.354,6039756,43,5,1:22.792,187.148,1,rosberg,German,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227
23310,23315,966,817,9,3,4,3,3,3,15.000,71,+20.858,6052260,53,1,1:21.134,190.972,1,ricciardo,Australian,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227
23311,23316,966,830,9,33,3,4,4,4,12.000,71,+21.323,6052725,66,6,1:22.887,186.934,1,max_verstappen,Dutch,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227
23312,23317,966,20,6,5,7,5,5,5,10.000,71,+27.313,6058715,61,2,1:22.497,187.817,1,vettel,German,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227
23313,23318,966,8,6,7,6,6,6,6,8.000,71,+49.376,6080778,47,3,1:22.512,187.783,1,raikkonen,Finnish,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227
23314,23319,966,807,10,27,5,7,7,7,6.000,71,+58.891,6090293,50,9,1:23.288,186.034,1,hulkenberg,German,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227
23315,23320,966,822,3,77,8,8,8,8,4.000,71,+1:05.612,6097014,65,11,1:23.540,185.472,1,bottas,Finnish,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227
23316,23321,966,13,3,19,9,9,9,9,2.000,71,+1:16.206,6107608,64,12,1:23.576,185.392,1,massa,Brazilian,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227
23317,23322,966,815,10,11,12,10,10,10,1.000,71,+1:16.798,6108200,62,13,1:23.607,185.324,1,perez,Mexican,2016,19,Mexican Grand Prix,30/10/2016,19:00:00,32,Autódromo Hermanos Rodríguez,Mexico,2227


In [40]:
df_circuits = df_circuits[['circuitId', 'name', 'country', 'alt']]
df_circuits.columns = ['circuitId', 'circuit_name', 'circuit_country', 'circuit_alt']
df = df.merge(df_circuits, on='circuitId', how='left', validate='m:1')

Output master dataset

In [41]:
df.to_csv('02_intermediate/pipeline_output.csv', index=False)